<a href="https://colab.research.google.com/github/MatiasJJ/ATML/blob/edits/Project/Project_Matias_Jakub_Madeleine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score

# Supervised learning
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.model_selection import KFold, train_test_split
import statsmodels.formula.api as smf
import statsmodels.api as sm

# Unsupervised learning
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, Normalizer
from scipy.optimize import linear_sum_assignment

# Project

Madeleine Ekblom, Matias Jääskläinen, Jakub Kubečka

Time used: many hours

In [0]:
# Read in the data

dummy = pd.read_csv("/content/sample_data/dummy.csv")
npf_train = pd.read_csv("/content/sample_data/npf_train.csv")
npf_test_hidden = pd.read_csv("/content/sample_data/npf_test_hidden.csv")

In [8]:
npf_train.head()

,id,date,event,partlybad,HYY_META.CO2168.mean,HYY_META.CO2168.std,HYY_META.CO2336.mean,HYY_META.CO2336.std,HYY_META.CO242.mean,HYY_META.CO242.std,HYY_META.CO2504.mean,HYY_META.CO2504.std,HYY_META.Glob.mean,HYY_META.Glob.std,HYY_META.H2O168.mean,HYY_META.H2O168.std,HYY_META.H2O336.mean,HYY_META.H2O336.std,HYY_META.H2O42.mean,HYY_META.H2O42.std,HYY_META.H2O504.mean,HYY_META.H2O504.std,HYY_META.H2O672.mean,HYY_META.H2O672.std,HYY_META.H2O84.mean,HYY_META.H2O84.std,HYY_META.NET.mean,HYY_META.NET.std,HYY_META.NO168.mean,HYY_META.NO168.std,HYY_META.NO336.mean,HYY_META.NO336.std,HYY_META.NO42.mean,HYY_META.NO42.std,HYY_META.NO504.mean,HYY_META.NO504.std,HYY_META.NO672.mean,HYY_META.NO672.std,HYY_META.NO84.mean,HYY_META.NO84.std,...,HYY_META.PAR.mean,HYY_META.PAR.std,HYY_META.PTG.mean,HYY_META.PTG.std,HYY_META.RGlob.mean,HYY_META.RGlob.std,HYY_META.RHIRGA168.mean,HYY_META.RHIRGA168.std,HYY_META.RHIRGA336.mean,HYY_META.RHIRGA336.std,HYY_META.RHIRGA42.mean,HYY_META.RHIRGA42.std,HYY_META.RHIRGA504.mean,HYY_META.RHIRGA504.std,HYY_META.RHIRGA672.mean,HYY_META.RHIRGA672.std,HYY_META.RHIRGA84.mean,HYY_META.RHIRGA84.std,HYY_META.RPAR.mean,HYY_META.RPAR.std,HYY_META.SO2168.mean,HYY_META.SO2168.std,HYY_META.SWS.mean,HYY_META.SWS.std,HYY_META.T168.mean,HYY_META.T168.std,HYY_META.T42.mean,HYY_META.T42.std,HYY_META.T504.mean,HYY_META.T504.std,HYY_META.T672.mean,HYY_META.T672.std,HYY_META.T84.mean,HYY_META.T84.std,HYY_META.UV_A.mean,HYY_META.UV_A.std,HYY_META.UV_B.mean,HYY_META.UV_B.std,CS.mean,CS.std
0,1,2000-01-23,nonevent,False,373.496585,0.189497,373.382593,0.172958,373.961481,0.235107,373.275062,0.165500,29.800885,22.892316,2.741951,0.253662,2.741975,0.238125,2.749136,0.256988,2.736543,0.226292,2.739259,0.211889,2.748902,0.257644,7.673882,18.320279,0.020366,0.028391,0.020617,0.029510,0.012716,0.031145,0.020741,0.032664,0.016790,0.031774,0.017439,0.029136,...,64.461794,34.106856,0.002187,0.005561,8.881671,6.029484,93.162439,0.736300,93.532469,0.693400,91.761728,0.672199,93.913827,0.774957,96.277654,0.933980,92.153537,0.718172,11.132826,6.353457,0.158171,0.122969,934.925926,1.517232,-9.811155,1.205806,-9.580934,1.243517,-9.941204,1.071755,-10.224472,0.965988,-9.651155,1.238891,2.677545,1.261612,0.044759,0.023748,0.000764,0.000048
1,2,2000-01-25,nonevent,False,381.752738,1.701439,381.392651,1.573735,382.373735,1.754803,380.980000,1.464652,18.389856,9.566668,2.313333,0.328202,2.310000,0.315463,2.308916,0.321506,2.320361,0.297872,2.342651,0.299325,2.312143,0.329732,29.663765,10.891472,0.483810,0.295051,0.494337,0.312692,0.404337,0.246385,0.488675,0.311784,0.421325,0.265687,0.435595,0.265177,...,39.684245,22.195953,0.009353,0.011258,5.547266,3.204514,95.802857,1.748414,95.578675,1.830929,93.989639,1.612301,95.268916,1.772817,97.426867,2.306357,94.580714,1.530332,7.345012,4.787854,2.050476,0.262710,888.285714,63.666293,-12.377122,1.930159,-12.144724,1.899699,-12.271055,1.774972,-12.399808,1.671699,-12.230048,1.918075,1.548098,0.804746,0.033443,0.020402,0.003630,0.000650
2,3,2000-02-13,nonevent,False,376.723579,0.468817,376.660842,0.466072,377.282447,0.498498,376.628105,0.461868,23.891954,13.275049,5.937684,0.143159,5.899053,0.142486,6.057340,0.132350,5.870000,0.149089,5.868737,0.144982,5.996632,0.138776,17.175105,10.329247,0.024526,0.033794,0.030842,0.035957,0.018723,0.039710,0.027895,0.039028,0.028000,0.039373,0.024526,0.036255,...,46.248851,26.678515,0.000000,0.000000,4.605295,2.383141,95.210316,3.447666,94.992526,3.539572,95.997553,3.213463,95.143263,3.644205,97.430316,3.727059,95.332737,3.347619,4.219557,2.818613,1.413263,0.316539,925.323529,15.328169,-0.039684,0.215015,0.119051,0.197613,-0.190570,0.217772,-0.523692,0.218085,0.080127,0.209476,1.644059,0.882067,0.023938,0.015601,0.005318,0.000479
3,4,2000-02-17,nonevent,False,378.600367,1.934180,378.464862,1.946536,379.785872,2.865022,378.316909,1.983430,23.795211,16.178905,5.710000,0.080335,5.715229,0.064873,5.795780,0.067058,5.688818,0.086743,5.662200,0.069218,5.748532,0.072748,18.218229,12.7204

In [0]:
X = npf_train.iloc[:,4:]
Y = npf_train.iloc[:,2]

Make Y categorical

In [0]:
labeler = LabelEncoder()
Y = labeler.fit_transform(Y.astype('str'))

In [44]:
list(labeler.classes_), set(Y)

(['II', 'Ia', 'Ib', 'nonevent'], {0, 1, 2, 3})

Reduce the number of dimensions

In [0]:
scaler = StandardScaler()
X = scaler.fit(X).transform(X)

In [26]:
X.shape

(724, 100)

In [0]:
pca = PCA(n_components=3)
npf_pca = pca.fit_transform(X)

In [28]:
npf_pca.shape, type(npf_pca)

((724, 3), numpy.ndarray)

In [0]:
npf_pca = pd.DataFrame(npf_pca, columns=["pca1","pca2","pca3"])

In [0]:
#px.scatter_3d(data_frame=npf_pca,x="pca1",y="pca2",z="pca3")

Split the train data into train and validation 

```X_train``` = training features  
```X_test``` = validation features  
```Y_train``` = training labels  
```Y_test``` = validation labels  

In [0]:
X_train,X_test,Y_train,Y_test = train_test_split(npf_pca,Y,random_state=42)

In [47]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((543, 3), (181, 3), (543,), (181,))

Try models

In [0]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

classifiers = [
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis()]

In [51]:
# iterate over classifiers
for name, clf in zip(names, classifiers):
    clf.fit(X_train, Y_train)
    score = clf.score(X_test, Y_test)
    print("{}:  {}".format(name,score))

Nearest Neighbors:  0.6077348066298343
Linear SVM:  0.6353591160220995
RBF SVM:  0.5524861878453039
Gaussian Process:  0.6408839779005525
Decision Tree:  0.6408839779005525
Random Forest:  0.6464088397790055
Neural Net:  0.6464088397790055
AdaBoost:  0.6022099447513812
Naive Bayes:  0.6132596685082873
QDA:  0.6022099447513812


Use cross-validation to find the best model

In [46]:
# k-fold cross-validation using sklearn 
kf = KFold(n_splits=10)

iterations=10
MSE_vals = []
MSE_outs = []
for iter in range(1,iterations):

  for model in [LinearRegression(), DecisionTreeRegressor(), RandomForestRegressor(), SVR(), Ridge()]:

    MSE_tr = []
    MSE_val = []
    MSE_out = []
    for train, val in kf.split(X_train):
        #print("%s %s" % (train, val))
        #print(train.shape, val.shape)
        fit = model.fit(X_train[train],Y_train[train])
        msetr = np.mean((fit.predict(X_train[train])-Y_train[train])**2)
        mseval = np.mean((fit.predict(X_train[val])-Y_train[val])**2)
        MSE_tr.append(msetr)
        MSE_val.append(mseval)
    #fit = model.fit(X_train,Y_train)
    #MSE_out=np.mean((fit.predict(X_te)-y_te)**2)
    #print("%21s: MSE_tr = %.2f MSE_val = %.2f MSE_in = %.2f MSE_out = %.2f" %
    #          (fit.__class__.__name__,
    #           np.mean(MSE_tr),
    #           np.mean(MSE_val),
    #           np.mean((fit.predict(X_tr)-y_tr)**2),
    #           MSE_out))
  MSE_vals.append(np.mean(MSE_val))
  MSE_outs.append(MSE_out)
print("Mean of validation loss is %.2f" % np.mean(MSE_vals))
print("Mean of test loss is %.2f" % np.mean(MSE_outs))

KeyError: ignored

Train the model 